# Model Building

transfer learning for improvements

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, classification_report
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
xgb.set_config(verbosity=0)
from sklearn.feature_selection import RFECV
import random
from sklearn.metrics import mean_squared_error
import sklearn.metrics as sm
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_absolute_error

In [2]:
# Drop first column of dataframe
def drop_first_col(df):
    return df.iloc[: , 1:]

train = pd.read_csv("dataset\\train.csv")
test = pd.read_csv("dataset\\test.csv")

train = drop_first_col(train)
test = drop_first_col(test)

## VAR (Multi Variate Time Series)

In [18]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

avg_rmse = []
avg_mae = []
for rid in train['RID'].unique():
    print('FOR RECORD',rid)
    
    VAR_train = (train.loc[train['RID']==rid]).drop(['RID','VISCODE'],axis=1)
    VAR_test = (test.loc[test['RID']==rid]).drop(['RID','VISCODE'],axis=1)
    
    if len(VAR_train) + len(VAR_test)>=6:

        model = VAR(endog=VAR_train)
        model_fit = model.fit(trend="nc")

        # make prediction on test set
        prediction = model_fit.forecast(model_fit.endog, steps=len(VAR_test))

        #converting predictions to dataframe
        pred = pd.DataFrame(index=range(0,len(prediction)),columns=[VAR_train.columns])
        for j in range(0,13):
            for i in range(0, len(prediction)):
                pred.iloc[i][j] = prediction[i][j]

        print("prediction :",'\n',pred['MMSE'],'\n')
        print("true value :",'\n',VAR_test['MMSE'],'\n')
        #check rmse
        rmse = np.sqrt(mean_squared_error(pred['MMSE'], VAR_test['MMSE']))
        mae = mean_absolute_error(VAR_test['MMSE'], pred['MMSE'])
        print('rmse value for MMSE is : ', rmse)
        print('MAE value for MMSE is : ', mae)
        avg_rmse.append(rmse)
        avg_mae.append(mae)

FOR RECORD 4.0
FOR RECORD 33.0
prediction : 
     MMSE
0   28.5
1  25.75 

true value : 
 2    25.0
3    25.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  2.531057091414539
MAE value for MMSE is :  2.1250000000000018
FOR RECORD 41.0
prediction : 
    MMSE
0  24.0
1  24.0 

true value : 
 4    24.0
5    23.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  0.7071067811865425
MAE value for MMSE is :  0.5000000000000018
FOR RECORD 42.0
prediction : 
         MMSE
0  17.321429
1  13.248724 

true value : 
 6    22.0
7    23.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  7.647757994606922
MAE value for MMSE is :  7.214923469387674
FOR RECORD 45.0
FOR RECORD 51.0
prediction : 
    MMSE
0  29.0
1  29.0 

true value : 
 9     28.0
10    26.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  2.2360679774998773
MAE value for MMSE is :  2.000000000000089
FOR RECORD 57.0
prediction : 
         MMSE
0  22.763158
1  22.682133 

true value : 
 11    20.0
12    14.0
Name

C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_

 
 20    29.0
21    29.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.2090515107122877
MAE value for MMSE is :  1.2083333333333783
FOR RECORD 126.0
prediction : 
         MMSE
0  32.333333
1  37.333333 

true value : 
 22    26.0
23    22.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  11.730776236512526
MAE value for MMSE is :  10.833333333333933
FOR RECORD 150.0
prediction : 
       MMSE
0    28.75
1  28.5625 

true value : 
 24    28.0
25    27.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.2255419719454983
MAE value for MMSE is :  1.1562500000000249
FOR RECORD 176.0
FOR RECORD 188.0
prediction : 
         MMSE
0  30.142857
1  30.193878 

true value : 
 28    28.0
29    29.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.7345288036703765
MAE value for MMSE is :  1.6683673469387763
FOR RECORD 190.0
FOR RECORD 204.0
prediction : 
    MMSE
0  24.0
1  24.0 

true value : 
 32    27.0
33    25.0
Name: MMSE, dtype: float64 

rmse value for MMSE i

C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_


rmse value for MMSE is :  1.7921510973005463
MAE value for MMSE is :  1.7083333333333321
FOR RECORD 291.0
prediction : 
    MMSE
0  29.0
1  29.5 

true value : 
 51    30.0
52    28.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.2747548783982114
MAE value for MMSE is :  1.2500000000000107
FOR RECORD 292.0
prediction : 
    MMSE
0  22.0
1  26.5 

true value : 
 53    22.0
54    22.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  3.181980515339876
MAE value for MMSE is :  2.2500000000004405
FOR RECORD 293.0
prediction : 
         MMSE
0     23.125
1  23.671875 

true value : 
 55    24.0
56    26.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.7586629886969662
MAE value for MMSE is :  1.6015625000000142
FOR RECORD 294.0
prediction : 
         MMSE
0  23.230769
1  28.183432 

true value : 
 57    26.0
58    23.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  4.155514761202245
MAE value for MMSE is :  3.9763313609467428
FOR RECORD 307.0
prediction : 

...

prediction : 
       MMSE
0    26.75
1  26.9375 

true value : 
 189    26.0
190    28.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  0.9196211855976416
MAE value for MMSE is :  0.9062500000000036
FOR RECORD 1045.0
prediction : 
       MMSE
0    29.25
1  29.3125 

true value : 
 191    25.0
192    29.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  3.0133167979819295
MAE value for MMSE is :  2.2812500000000213
FOR RECORD 1046.0
prediction : 
         MMSE
0  27.666667
1  27.333333 

true value : 
 193    27.0
194    28.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  0.6666666666666963
MAE value for MMSE is :  0.6666666666666963
FOR RECORD 1073.0
prediction : 
     MMSE
0   28.5
1  23.25 

true value : 
 195    24.0
196    21.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  3.557562367689411
MAE value for MMSE is :  3.375000000000023
FOR RECORD 1077.0
prediction : 
    MMSE
0  28.0
1  28.0 

true value : 
 197    27.0
198    28.0
Name: MMSE, dtype: flo

C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_

 

rmse value for MMSE is :  2.6279745052035968
MAE value for MMSE is :  2.6250000000000178
FOR RECORD 1246.0
prediction : 
    MMSE
0  30.0
1  30.0 

true value : 
 222    30.0
223    28.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.4142135623730876
MAE value for MMSE is :  0.9999999999999947
FOR RECORD 1247.0
FOR RECORD 1260.0
prediction : 
       MMSE
0    27.25
1  27.0625 

true value : 
 226    26.0
227    26.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.1600444495794073
MAE value for MMSE is :  1.1562499999999947
FOR RECORD 1265.0
prediction : 
         MMSE
0  28.666667
1  28.555556 

true value : 
 228    27.0
229    27.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.612068680949626
MAE value for MMSE is :  1.6111111111110805
FOR RECORD 1268.0
prediction : 
    MMSE
0  29.0
1  29.0 

true value : 
 230    30.0
231    29.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  0.7071067811865425
MAE value for MMSE is :  0.5000000000000036
FOR

C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_

 1423.0


In [19]:
print('AVG RMSE:',sum(avg_rmse) / len(avg_rmse))
print('AVG MAE:',sum(avg_mae) / len(avg_mae))

AVG RMSE: 2.6470164258946647
AVG MAE: 2.3747729466502046
